In [2]:
import sys
from pathlib import Path
import time
import numpy as np
from pprint import pprint as pp
import matplotlib as mpl
from matplotlib import pyplot as plt
plt.ion()
#plt.style.use('seaborn-v0_8-muted')

from IPython.display import Markdown
# To get multiple outputs into 1 cell w/o using print:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# autoreload extension
from IPython import get_ipython

ipython = get_ipython()
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

# -----------------------------------------
# TWO USEFUL FUNCTIONS:

def add_to_sys_path(this_path, up=False):
    """
    Prepend this_path to sys.path.
    If up=True, path refers to parent folder (1 level up).
    """
    if up:
        newp = Path(this_path).parent
    else:
        newp = Path(this_path)
    src = newp.joinpath("src")
    if src.exists():
        newp = str(src)
    else:
        newp = str(newp)
    if newp not in sys.path:
        sys.path.insert(1, newp)
        print('Path added to sys.path: {}'.format(newp))


# Filtered dir() for method discovery:
def fdir(obj, start_with_str='_', exclude=True):
    return [d for d in dir(obj) if not d.startswith(start_with_str) == exclude]


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Insert current src dir into sys.path so that modules in ../src can be imported:
# CHANGE THIS IF NEEDED:

add_to_sys_path(Path.cwd(), up=True)

Path added to sys.path: /home/cat/projects/mcce_dev/src


---
# Test executable module

In [7]:
from functools import partial
import subprocess

import ms_sampling_to_pdbs as sampling

load_npz = partial(np.load, allow_pickle=True)
save_npz = partial(np.savez, allow_pickle=True)

In [8]:
HERE = Path.cwd()  # do not change

test_folder = "granepura_GunnerLab/mcce_data" # you can change this to any other mcce folder
mcce_dir = HERE.parent.parent.joinpath(test_folder)  # do not change
mcce_dir, mcce_dir.exists()

(PosixPath('/home/cat/projects/granepura_GunnerLab/mcce_data'), True)

In [11]:
pH = 7.0
Eh= 0.0
overwrite = False

ms = sampling.MS(mcce_dir,pH, Eh, overwrite_split_files = overwrite)
ms.T, ms.MC_RUNS

	Getting runprm data.
	Getting conformers data.
	Getting header data.


(298.15, 6)

In [12]:
ms.get_mc_data(0)
print(f"{ms.counts = :,}; {ms.MC_RUNS = :,}")

print(f"{len(ms.microstates) = :,}")

	Getting MC data from .npz file for MC0
ms.counts = 1,500,000; ms.MC_RUNS = 6
len(ms.microstates) = 515,400


In [13]:
#which = msout_dir.joinpath("pH6eH0ms")
!ls -l {ms.msout_file_dir}

total 231068
-rw-r--r-- 1 cat cat   9742123 Nov 16 17:44 MC0
-rw-r--r-- 1 cat cat 178231770 Nov 16 17:45 MC0.npz
-rw-r--r-- 1 cat cat   9725008 Nov 16 17:44 MC1
-rw-r--r-- 1 cat cat   9705947 Nov 16 17:44 MC2
-rw-r--r-- 1 cat cat   9722617 Nov 16 17:44 MC3
-rw-r--r-- 1 cat cat   9708327 Nov 16 17:44 MC4
-rw-r--r-- 1 cat cat   9727680 Nov 16 17:44 MC5
-rw-r--r-- 1 cat cat      4440 Nov 16 17:44 header
drwxr-xr-x 2 cat cat      4096 Nov 16 19:15 pdbs_from_ms
-rw-r--r-- 1 cat cat      5002 Nov 16 19:15 smsm0_3_dc.npz
-rw-r--r-- 1 cat cat      5003 Nov 16 19:21 smsm0_3_de.npz
-rw-r--r-- 1 cat cat      4973 Nov 16 19:21 smsm0_3_r.npz


In [14]:
print(ms)
ms.mcce_out
ms.msout_file_dir
ms.msout_fpath
ms.msout_fpath.name

MS('/home/cat/projects/granepura_GunnerLab/mcce_data', 7.0, 0.0, overwrite_split_files=False)


PosixPath('/home/cat/projects/granepura_GunnerLab/mcce_data')

PosixPath('/home/cat/projects/granepura_GunnerLab/mcce_data/ms_out/pH7eH0ms')

PosixPath('/home/cat/projects/granepura_GunnerLab/mcce_data/ms_out/pH7eH0ms.txt')

'pH7eH0ms.txt'

# Check output of  sampling

In [17]:
ms.free_residue_names[:10]

['META0001_',
 'TYRA0004_',
 'ASNA0006_',
 'GLUA0008_',
 'THRA0010_',
 'TYRA0011_',
 'TRPA0013_',
 'ARGA0019_',
 'ASNA0022_',
 'SERA0024_']

In [27]:
n_sample_size = 4
sample_kind =  "deterministic"
ms_sort_by = "energy"
save = True

info, selection_energies, smsm = sampling.get_smsm(ms, n_sample_size, sample_kind, ms_sort_by, save_to_npz=save)
info

	Sampling microstates (kind: deterministic) for MC0.


["size=4, kind='deterministic', sort_by='energy', reverse=False, seed=None",
 'MC=0',
 '(145, 4)',
 '20-Nov-23 15:38:37',
 PosixPath('/home/cat/projects/granepura_GunnerLab/mcce_data/ms_out/pH7eH0ms/smsm0_4_de.npz')]

In [29]:
fname = sampling.get_output_filename(ms.selected_MC,
                                     sample_kind,
                                     ms_sort_by,
                                     False,
                                     size=n_sample_size)
d_npz_file = ms.msout_file_dir.joinpath(fname)
smsm_data = load_npz(d_npz_file)
info_dict = sampling.smsm_data_info_to_dict(smsm_data["info"])
info_dict

In [77]:
len("MET01A0001_002")
xx = 467.14
repr(xx)
sx = f"{xx:,.2f}"
sx

14

'467.14'

'467.14'

In [83]:
sel_indices = smsm_data["selection_indices"]
sel_energies = smsm_data["selection_energies"]
res_names = smsm_data["residue_names"]
smsm = smsm_data["smsm"]

#hdr = f"RESIDUE   | " + "".join([f"{x:<15}" for x in sel_indices])
#hdr = f"RESIDUE   | " + " ".join([f"{x:>14,.2f}" for x in sel_energies])
#print(hdr)
#print("_" * len(hdr))

for r in range(smsm.shape[0]):
    rc = smsm[r,:]
    row_confs = ms.confnames_by_iconfs(rc.tolist())

    print(f"{res_names[r]} |", *row_confs)
    if r > 5:
       break


META0001_ | MET01A0001_002 MET01A0001_007 MET01A0001_007 MET01A0001_002
TYRA0004_ | TYR01A0004_001 TYR01A0004_001 TYR01A0004_001 TYR01A0004_001
ASNA0006_ | ASN01A0006_001 ASN01A0006_001 ASN01A0006_001 ASN01A0006_001
GLUA0008_ | GLU-1A0008_010 GLU-1A0008_010 GLU-1A0008_010 GLU-1A0008_010
THRA0010_ | THR01A0010_001 THR01A0010_001 THR01A0010_002 THR01A0010_002
TYRA0011_ | TYR01A0011_004 TYR01A0011_003 TYR01A0011_010 TYR01A0011_003
TRPA0013_ | TRP01A0013_005 TRP01A0013_004 TRP01A0013_003 TRP01A0013_007


In [85]:
smsm[:5,:5]
res_names[:5]

array([[ 1,  6,  6,  1],
       [20, 20, 20, 20],
       [24, 24, 24, 24],
       [36, 36, 36, 36],
       [38, 38, 39, 39]])

array(['META0001_', 'TYRA0004_', 'ASNA0006_', 'GLUA0008_', 'THRA0010_'],
      dtype='<U9')

# RESUME HERE

In [ ]:
def smsm_to_full_confids(smsm_data):
    """Convert smsm matrix numbers (iconfs) to confIDs and add the
    fixed residues in each selected state.
    """

    sel_indices = smsm_data["selection_indices"]
    sel_energies = smsm_data["selection_energies"]
    res_names = smsm_data["residue_names"]
    smsm = smsm_data["smsm"]

    #hdr = f"RESIDUE   | " + "".join([f"{x:<15}" for x in sel_indices])
    #hdr = f"RESIDUE   | " + " ".join([f"{x:>14,.2f}" for x in sel_energies])
    #print(hdr)
    #print("_" * len(hdr))

    for r in range(smsm.shape[0]):
        rc = smsm[r,:]
        row_confs = ms.confnames_by_iconfs(rc.tolist())
        print(f"{res_names[r]} |", *row_confs)

    
    for i, c in enumerate(range(smsm.shape[1])):
        rc = smsm[:,i]
    
        # add fixed res
        all_iconfs = sorted(ms.fixed_iconfs + rc)
        selected_confs = ms.confnames_by_iconfs(all_iconfs)
        print(f"{len(selected_confs)= }")
    
    
        if c == 0:
            break
        # write pdb with one_pdb_confs; ms_index from smsm row 1
        #ms_to_pdb(selected_confs: list, ms_index: int, mc_run: int, remark_data: str, step2_path: str, output_folder: str)


In [108]:
!ls -l {ms.msout_file_dir}

total 108480
-rw-r--r-- 1 cat cat 74656121 Nov 13 14:14 MC0.npz
-rw-r--r-- 1 cat cat  7277473 Nov 13 14:14 MC1
-rw-r--r-- 1 cat cat  7272764 Nov 13 14:14 MC2
-rw-r--r-- 1 cat cat  7300876 Nov 13 14:14 MC3
-rw-r--r-- 1 cat cat  7283815 Nov 13 14:14 MC4
-rw-r--r-- 1 cat cat  7273831 Nov 13 14:14 MC5
-rw-r--r-- 1 cat cat     1372 Nov 13 14:14 header
-rw-r--r-- 1 cat cat     3555 Nov 13 18:07 smsm0_de.npz


In [109]:
n_sample_size = 4
sample_kind =  "deterministic"
ms_sort_by = "energy"
save = True
out_dir = Path.cwd()

sampling.pdbs_from_smsm(ms,
                        n_sample_size,
                        sample_kind,
                        sort_by = ms_sort_by,
                        sort_reverse = False,
                        seed = None,
                        output_pdb_format = "standard",
                        output_dir = out_dir,
                        clear_pdbs_folder = False,
                        list_files = True
                       )


Creating n=4 MCCE_PDB files in /home/cat/projects/mcce_dev/notebooks from (n) microstates sorted by 'energy'.

PDB files creation over.
Files in /home/cat/projects/mcce_dev/notebooks/pdbs_from_ms:

	 /home/cat/projects/mcce_dev/notebooks/pdbs_from_ms/mc0_ms104828.pdb
	 /home/cat/projects/mcce_dev/notebooks/pdbs_from_ms/mc0_ms211882.pdb
	 /home/cat/projects/mcce_dev/notebooks/pdbs_from_ms/mc1_ms211972.pdb
	 /home/cat/projects/mcce_dev/notebooks/pdbs_from_ms/mc2_ms3.pdb
	 /home/cat/projects/mcce_dev/notebooks/pdbs_from_ms/mc2_ms211759.pdb
	 /home/cat/projects/mcce_dev/notebooks/pdbs_from_ms/mc0_ms320499.pdb
	 /home/cat/projects/mcce_dev/notebooks/pdbs_from_ms/mc2_ms104929.pdb
	 /home/cat/projects/mcce_dev/notebooks/pdbs_from_ms/mc0_ms4.pdb
	 /home/cat/projects/mcce_dev/notebooks/pdbs_from_ms/mc2_ms320467.pdb
	 /home/cat/projects/mcce_dev/notebooks/pdbs_from_ms/mc2_ms4.pdb
	 /home/cat/projects/mcce_dev/notebooks/pdbs_from_ms/mc2_ms211972.pdb
	 /home/cat/projects/mcce_dev/notebooks/pdbs_fr

In [112]:
# Inspect a pdb head:

!head -n 50 ./pdbs_from_ms/mc0_ms104828.pdb


REMARK 250
REMARK 250 EXPERIMENTAL DETAILS
REMARK 250   EXPERIMENT TYPE               : MCCE simulation
REMARK 250   DATE OF DATA COLLECTION       : 13-Nov-23
REMARK 250   REMARK: Date of data collection is the date this pdb was created.
REMARK 250 SAMPLING DETAILS
REMARK 250   SAMPLING PARAMETERS           : size=4, kind='deterministic', sort_by='energy', reverse=False
REMARK 250   MATRIX FILE NAME              : [..]/4lzt/ms_out/pH6eH0ms/smsm0_de.npz
REMARK 250   MATRIX SHAPE                  : (65, 4)
REMARK 250   MATRIX CREATION DATE          : 13-Nov-23 18:07:39
REMARK 250   MICROSTATE SELECTION INDEX    : 104,828
REMARK 250   MICROSTATE CREATION INDEX     : 104,828
REMARK 250   SELECTED MICROSTATE ENERGY    : 195.46 (kcal/mol)
REMARK 250 EXPERIMENTAL CONDITIONS
REMARK 250   TEMPERATURE                   : 298.15 (K)
REMARK 250   PH                            : 6.00
REMARK 250   EH                            : 0.00
REMARK 250   METHOD                        : MONTERUNS
REMARK 250

---
# TODO? PDB conversions

## name.txt
> Rename rule file contains rules to format atom names and residue names.
 The purpose is to unify residue names to 3-char mcce names
 and break some big cofactors into small ones.
 Each line has two fields separated by at least one space. Each field
 is 14 characters long, matching the atom, residue name, chainID and
 sequence number field of a pdb line. The first string will be
 replaced by the second string.
 Symbol "*" in the first string is a wildcard that matchs any character.
 It means "do not replace" in the second string.
 The replace is accumulative in the order of appearing in this file.


The first 3 lines:
```python
0123456789012  01234567890123
***** *******  *****_********
****** *****   ******_*******
******* ***  > *******_****
```
mean: replace a space in position 5, 6 or 7 by "_".

**

In [149]:
nametxt = mcce_dir.joinpath("name.txt")
nametxt

# use when running subprocesses from this nbk
nametxt2 = nametxt.relative_to(Path.cwd().parent)
nametxt2

PosixPath('/home/cat/projects/mcce_dev/data/4lzt/name.txt')

PosixPath('data/4lzt/name.txt')

In [93]:
import re
from collections import defaultdict


regex = r'^\*{5,8}\s\*{5,}\s\s'
split_tag = "*@@"

mcce_to_pdb_name_dict = defaultdict(list)
with open(nametxt) as f:
    for i, line in enumerate(f):
        if (line.startswith("#")
            or line.startswith(" D")
            or re.match(regex, line)
           ):
            continue
        if len(line) <29:
            continue

        line = line[:30].replace("*  ", split_tag)
        from_str, to_str = line.split(split_tag)[:2]
        mcce_to_pdb_name_dict[to_str].append(from_str)


In [92]:
#pp(mcce_to_pdb_name_dict)

In [90]:
s = "***** NA******  *****_NA******"
x = "012345678901234567890123456789"
len(s), len(x)

(30, 30)